# 2025 Rewrite

Because this shouldn't need 2 notebooks (it's too confusing).

I mostly tried to preserve the original data analysis code. This should work if you run it from start to finish.

The main thing you need to change is the 'year' and the fields in the Data Cleaning section as well as `years` in the combining section.

In [188]:
import pandas as pd
import numpy as np

from jsonmerge import merge
import json
import pprint

## Generate current year candidate data

In [189]:
year = '2025' # change me

### Data Cleaning

In [190]:
student_reg_csv = f'./{year}/Student Registration-Candidate Database.csv'
student_signin_csv = f'./{year}/Student Sign In-Grid view.csv'

df_reg = pd.read_csv(student_reg_csv, header=0)
df_signin = pd.read_csv(student_signin_csv, header=0)

In [ ]:
df = pd.DataFrame()

# Set up
signin_cols = ['School', 'Major', 'Degree']
reg_cols = ['School', 'Major Field of Study', 'Program']

fieldString = 'Major'
degreeFieldString = 'Degree'

# Left join
df = df_signin.merge(df_reg[['Email'] + reg_cols], on='Email', how='left', suffixes=('', '_reg'))
reg_col_names = [col2 + ('' if col1 != col2 else '_reg') for col1, col2 in zip(signin_cols, reg_cols)]

# For each column, fill NaNs in df_signin with values from df_reg
for col1, col2 in zip(signin_cols, reg_col_names):
    df[col1] = df[col1].fillna(df[col2])

# Drop the extra *_reg columns if you don't need them anymore
df = df.drop(columns=reg_col_names)
print("Fraction of randos that didn't sign up", df['School'].isna().mean())



# antijoin
df_extra = df_reg[~df_reg['Email'].isin(df_signin['Email'])]
print(f"People that didn't show up/people reg = {df_extra.shape[0]} / {df_reg.shape[0]}")
print('No of sign ups:' df_reg.shape[0])

Fraction of randos that didn't sign up 0.1511627906976744
People that didn't show up/people reg = 2019 / 2682
{'Year': '2025', 'Number of Attendees': 2682}


### Field of Study

In [192]:
print('Unique majors:', df[fieldString].nunique())


names = df[fieldString].dropna().unique()
names_cs = [n for n in names if n.find('Computer Sciences') != -1]
names_eng = [n for n in names if n.find('Engineering') != -1]

print('Here are some sample majors grouped under computer science')
names_cs

Unique majors: 98
Here are some sample majors grouped under computer science


['Computer Sciences, Artificial Intelligence (AI) & Machine Learning (ML)',
 'Computer Sciences, Computer Science',
 'Computer Sciences, Data Science',
 'Computer Sciences, Web Design/Development',
 'Computer Sciences, Information Technology',
 'Computer Sciences, Computer Programming',
 'Computer Sciences, Information Systems/Technology Management',
 'Computer Sciences, Information Sciences & Systems',
 'Computer Sciences, Management & Information Systems',
 'Computer Sciences, Software Development',
 'Computer Sciences, Hardware Technology',
 'Computer Sciences, Data Analytics',
 'Computer Sciences, Network Security & Cyber Intelligence',
 'Computer Sciences, Combinatorics & Optimization',
 'Computer Sciences, Computer Systems Analysis',
 'Computer Sciences, Computer Networking/Administration']

Now, we will summarize the majors into this attribute called `mostRelevantField`

In [193]:
# In this list, the last items has the priority to override
fieldRanking = ['marketing', 'industrial', 'supply chain', 'law', 'chemical', 'finance', 'economic',
     'political', 'business', 'management', 'mba', 'civil', 'physics', 'mechanical', 'material', 'aerospace',
     'electrical', 'electronic', 'computer', 'bio', 'analytics', 'data', 'artificial', 'machine learning']


#create new column
mostRelevantField = ["Other"]*len(df)

# for each candidate (row)
for i in range(len(df)): 
    # get candidate's fields of study
    fieldsOfCandidate = df.loc[i, fieldString]
    
    if isinstance(fieldsOfCandidate, str):
        # make lowercase if it is a string! 
        fieldsOfCandidate = fieldsOfCandidate.lower()
        
        # for each field (column) in the ranking list
        for wantedColumn in fieldRanking: 

            if wantedColumn in fieldsOfCandidate:
                mostRelevantField[i] = wantedColumn


# for similar fields, use a synonym
fieldSynonyms = {
    "artificial": "AI",
    "machine learning": "AI",
    
    "data": "Data Science",
    "analytics": "Data Science",
    
    "bio": "Biology",
    
    "computer": "CS",
    
    "aerospace": "Aero",
    
    "material": "Materials",
    
    "mechanical": "Mechanical Eng",
    
    "electrical": "Electrical Eng",
    "electronic": "Electrical Eng",
    
    "physics": "Physics",
    
    "civil": "Civil Eng",
    
    "business": "Business",
    "mba": "Business",
    "management": "Business",
    
    "political": "Policital Sci",
    
    "economic": "Finance",
    "finance": "Finance",
    
    "chemical": "Chemistry",
    
    "supply chain": "Industrial Eng",
    "industrial": "Industrial Eng",
    
    "law": "Law",
    
    "marketing": "Marketing",
    
}


replacementCount = 0

# for each candidate
for i in range(len(mostRelevantField)):

    # if the candidate's field is in fieldSynonyms
    if mostRelevantField[i] in fieldSynonyms:

        # replace it with the more common field name
        mostRelevantField[i] = fieldSynonyms[mostRelevantField[i]]
        
        replacementCount = replacementCount+1
      
print(str(replacementCount) + " fields have been renamed!")
df["mostRelevantField"] = mostRelevantField

563 fields have been renamed!


### University Name

In [194]:
list(set(df['School']))[:10]

['Columbia',
 'emerson',
 'University of Dayton',
 'Brown University',
 'Broad Institute of MIT and Harcard',
 'Walden University',
 'Bard',
 'Franklin Olin College of Engineering',
 'University of Illinois at Urbana-Champaign',
 'KU Leuven']

In [195]:
# In this list, the last items has the priority to override
uniRanking = ['university of connecticut',
              'worcester polytechnic', 'wpi',
              'wellesley college',
              'university of massachusetts boston',
              'dartmouth', 'dartmouth college', 'dartmouth university',
              'brandeis', 'brandeis university',
              'university of massachusetts amherst',
              'boston college',
              'tufts', 'tufts university',
              'new york university', 'nyu',
              'hult international', 'hult business',
              'boston university',
              'northeastern university', 'northeastern',
              'harvard', 'harvard college', 'harvard university',
              'massachusetts institute of technology', 'mit', 'smith college']


#create new column
mostRelevantUni = ["Other"]*len(df)

# for each candidate (row)
for i in range(len(df)): 
    # get candidate's fields of study
    unisOfCandidate = df.loc[i, "School"]
    
    if isinstance(unisOfCandidate, str):
        # make lowercase if it is a string! 
        unisOfCandidate = unisOfCandidate.lower()
        
        # for each field (column) in the ranking list
        for wantedUniversity in uniRanking: 
            if wantedUniversity in unisOfCandidate:
                mostRelevantUni[i] = wantedUniversity



# For equivalent university names, use a single (unique) name

uniSynonyms = {
    "massachusetts institute of technology": "MIT",
    "mit": "MIT",
    
    "harvard": "Harvard",
    "harvard college": "Harvard",
    "harvard university": "Harvard",
    
    "hult international": "Hult Business",
    "hult business": "Hult Business",
    
    "northeastern university": "Northeastern",
    "northeastern": "Northeastern",
    
    "boston university": "BU",
    
    "tufts": "Tufts",
    "tufts university": "Tufts",
    
    "dartmouth": "Dartmouth",
    "dartmouth university": "Dartmouth",
    "dartmouth college": "Dartmouth",
    
    "new york university": "NYU",
    "nyu": "NYU",
        
    "boston college": "Boston College",
    
    "brandeis": "Brandeis",
    "brandeis university": "Brandeis",
    "wellesley": "Wellesley",
    "wellesley college": "Wellesley",
    
    "university of massachusetts amherst": "UMass Amherst",
    
    "university of massachusetts boston": "UMass Boston",
    
    
    "worcester polytechnic": "WPI",
    "wpi": "WPI",
    
    "university of connecticut": "UConn",
    
    "smith college": "Smith"
    
}




replacementCount = 0

# for each candidate
for i in range(len(mostRelevantUni)):

    # if the candidate's field is in fieldSynonyms
    if mostRelevantUni[i] in uniSynonyms:

        # replace it with the more common field name
        mostRelevantUni[i] = uniSynonyms[mostRelevantUni[i]]
        
        replacementCount = replacementCount+1
      
print(str(replacementCount) + " university names have been renamed!")
df["mostRelevantUni"] = mostRelevantUni

683 university names have been renamed!


### Highest Degree

In [196]:
# List unique degree names
list(set(df[degreeFieldString]))[:10]

['Postgraduate Internship',
 'Advanced Study Program',
 'MBA',
 'Staff',
 'Undergraduate',
 'Graduate Student',
 'alumni',
 'Gap year',
 'Executive Sloan Fellow MBA',
 'Postdoctoral']

In [197]:
degreeRanking = ['high school', 'undergraduate','bachelor',
     'master', 'mba', 'phd', 'doctorate',
     'postdoc',  'post doc', 'md']




#create new column
highestDegree = ["Other"]*len(df)

# for each candidate (row)
for i in range(len(df)): 
    
    # get candidate's fields of study
    degreeOfCandidate = df.loc[i, degreeFieldString]
    
    if isinstance(degreeOfCandidate, str):
        # make lowercase if it is a string! 
        degreeOfCandidate = degreeOfCandidate.lower()
        
        # for each field (column) in the ranking list
        for preferredDegree in degreeRanking: 

            if preferredDegree in degreeOfCandidate:
                highestDegree[i] = preferredDegree



degreeSynonyms = {
    "high school": "High school",
    
    "undergraduate": "Bachelors",
    "bachelor": "Bachelors",
    
    "master": "Masters",
    
    "mba": "MBA",
    
    'phd': "PhD",
    'doctorate': "PhD",
    
    "postdoc": "Postdoc",
    "post doc": "Postdoc",
    
    'md': "MD"
}





replacementCount = 0

# for each candidate
for i in range(len(highestDegree)):

    # if the candidate's field is in degreeSynonyms
    if highestDegree[i] in degreeSynonyms:

        # replace it with the more common name
        highestDegree[i] = degreeSynonyms[highestDegree[i]]
        
        replacementCount = replacementCount+1
      
print(str(replacementCount) + " degree names have been renamed!")




df["highestDegree"] = highestDegree

773 degree names have been renamed!


### Clean up

In [198]:
df_extra['School']

0                                 MIT
1                                 MIT
2                   Boston University
3               University at Buffalo
4       Washington and lee university
                    ...              
2671                            Tufts
2678                Boston University
2679                              JHU
2680                     Northeastern
2681                          Harvard
Name: School, Length: 2019, dtype: object

In [199]:
columnsToStay = ['Email',
                 'mostRelevantField',
                 'mostRelevantUni',
                 'highestDegree']

cols2group = ['mostRelevantUni', 'mostRelevantField', 'highestDegree']
groupedCandidateNumbers = pd.DataFrame({'count': df[columnsToStay].groupby(cols2group).size()})

#Source: https://stackoverflow.com/a/10374456

groupedCandidateNumbers = groupedCandidateNumbers.reset_index().set_index('mostRelevantUni')
print("\nNew index: \n")
print(set(groupedCandidateNumbers.index))
print(groupedCandidateNumbers.head())


New index: 

{'Tufts', 'UConn', 'Harvard', 'BU', 'NYU', 'MIT', 'Northeastern', 'UMass Boston', 'Boston College', 'Other', 'Dartmouth', 'WPI', 'Hult Business', 'UMass Amherst'}
                mostRelevantField highestDegree  count
mostRelevantUni                                       
BU                        Biology     Bachelors      4
BU                        Biology       Masters      1
BU                        Biology         Other      2
BU                       Business     Bachelors      3
BU                       Business           MBA      2


In [200]:
# Change uni name order
# May need to delete some unis that didn't show up
#Wellesley left out 
new_index = ["MIT", "Harvard", "NYU", "Hult Business",
             "Northeastern", "BU", "Tufts", "Dartmouth",
             #"Brandeis",
             "Boston College", 
             "UMass Amherst", "UMass Boston", "WPI", "UConn", "Other"
             #"UMass Amherst", "UMass Boston", "WPI", "Other"
            ]

new_index.reverse()

# Show the difference between sets
print('Missing (comment these out): ', list(set(new_index) - set(groupedCandidateNumbers.index)))


groupedCandidateNumbersRelisted = groupedCandidateNumbers.loc[new_index]
print(set(groupedCandidateNumbersRelisted.index))
groupedCandidateNumbersRelisted = groupedCandidateNumbersRelisted.reset_index()
groupedCandidateNumbersRelisted.rename(columns={
    'mostRelevantUni': 'University',
    'mostRelevantField': 'Major',
    'highestDegree': 'Highest degree',
    'count': 'Number of Records'}, 
    inplace=True)


# Add year information as a new column
# Applied to all rows
listCurrentYear = groupedCandidateNumbersRelisted
listCurrentYear['Year'] = year

Missing (comment these out):  []
{'Tufts', 'UConn', 'Northeastern', 'BU', 'NYU', 'Harvard', 'MIT', 'UMass Boston', 'Boston College', 'Other', 'Dartmouth', 'WPI', 'Hult Business', 'UMass Amherst'}


### Export as CSV and JSON

In [201]:
# Export as CSV
df.to_csv(f"./{year}/FP_candidates_{year}.csv")
print(f'Wrote to "./{year}/FP_candidates_{year}.csv"')

# Export as JSON
listCurrentYear.to_json(f"./{year}/FP_Groups_{year}.json", orient='records')
print(f'Wrote to "./{year}/FP_Groups_{year}.json"')

Wrote to "./2025/FP_candidates_2025.csv"
Wrote to "./2025/FP_Groups_2025.json"


The end

## Combine data

In [202]:
years = ['2020', '2021', '2022', '2024', '2025']
outfile = "../docs/data/FP_Candidate_Groups_All.json"

def cat_json(outfile, years):
    file = open(outfile, "w")
    file.write("[%s]" % (",".join([mangle(file("FP_Groups_" + y + ".json").read()) for y in years])))

def mangle(s):
    return s.strip()[1:-1]

In [204]:
jsons = []
for idx, y in enumerate(years):
    with open(f"./{y}/FP_Groups_{y}.json", "r") as jsonFile:
        currentJson = jsonFile.read()
        jsons.append(mangle(currentJson))
jsonList = "[" + ','.join(jsons) + ']'

file = open(outfile, "w")
file.write(jsonList)
print(f'Wrote data from {len(years)} years to {outfile}')

Wrote data from 5 years to ../docs/data/FP_Candidate_Groups_All.json
